In [1]:
# Load libraries
import pandas as pd
import datetime as dt

In [2]:
# Load files: dividends_latest.csv, earnings_latest.csv
dividends_data = pd.read_csv("data/kaggle/dividends_latest.csv") 
earnings_data  = pd.read_csv("data/kaggle/earnings_latest.csv") 

In [20]:
# Function to generate csv for given dataframe.
def df_to_csv(df, dividends_df, earnings_df, csv):

    # Join files: dividends and earnings
    result = df.copy()

    # Join with dividends
    result = pd.merge(result,
                     dividends_df[['date', 'dividend']],
                     on='date', how='left')

    result['ex_dividend_date'] = (result['dividend'] >= 0)

    # Join with earnings
    result = pd.merge(result,
                     earnings_df[['date','qtr','eps_est','eps','release_time']],
                     on='date', how='left')
    result['earnings_date'] = (result['qtr'].notna())
    result = result.drop(columns=['qtr','release_time'])
    
    # Create new columns
    result['Next Open']  = result['open'].shift(1)
    result['Next Close'] = result['close'].shift(1)
    result['Prev Open']  = result['open'].shift(-1)
    result['Prev Close'] = result['close'].shift(-1)

    # Create column: Quarter
    def date_to_quarter(date):
        year,month,day = date.split('-')
        return pd.Timestamp(dt.date(int(year),int(month),int(day))).quarter

    result['Quarter'] = result['date'].apply(lambda x: date_to_quarter(x))
    
    # Save to csv
    result.to_csv(csv)

In [26]:
# Function to merge all datasets and save to csv.
# Output:
# - data/<STOCK>_full.csv
# - data/<STOCK>_2014_to_2019.csv
def merge_and_save(symbol, price_file):
   
    # Set output file names
    out_five = "data/" + symbol.lower() + "_2014_to_2019.csv"
    out_full = "data/" + symbol.lower() + "_full.csv"
    
    # Get dividends/earnings for stock.
    dividends = dividends_data.loc[dividends_data['symbol'] == symbol.upper()]
    earnings  = earnings_data.loc[earnings_data['symbol'] == symbol.upper()]
    
    # Get price data.
    price_data = pd.read_csv(price_file) 
    price_data_2014_to_2019 = price_data.loc[price_data['date'] > "2014-08-09"]

    # Create 5-year csv.
    df_to_csv(price_data_2014_to_2019, dividends, earnings, out_five)

    # Create full csv.
    df_to_csv(price_data, dividends, earnings, out_full)

In [27]:
merge_and_save("intc", "data/kaggle/intc.csv")

In [28]:
names = ("amd","asml","mu","nvda","tsm","txn")
for name in names:
    merge_and_save(name, "data/kaggle/"+ name + ".csv")